# [KDD Cup|Humanities Track Tutorial Q-Learning](https://compete.hexagon-ml.com/tutorial/kdd-cuphumanities-track-tutorial/)

### KDD Cup|Humanities Track Tutorial Q-Learning
This Tutorial builds on the previous tutorial to demonstrate a baseline implementation of a standard Reinforcement Learning (RL) Algorithm

### State

$S \in \{1,2,3,4,5\}$

### Action
$A_S = [a_{ITN},a_{IRS}]$

where  $a_{ITN} \in [0,1]$ and $a_{IRS} \in [0,1]$

### Reward
$R_{\pi} \in (- \infty,\infty)$

![](image/rewards2.png)

In [1]:
import numpy as np
from collections import defaultdict
import random

from netsapi.challenge import * 

### Learning a Value Function Based on ϵ-greedy action selection

This common resource was used as a reference for the implementation presented here: https://kofzor.github.io/Learning_Value_Functions/. Please refer to the blog and this Tutorial in tandem. The code below uses the first example from the blog with the Challenge Environment (as opposed to Gym).

In [2]:
env = ChallengeSeqDecEnvironment()

Q = defaultdict(lambda : 0.) # Q-function
n = defaultdict(lambda : 1.) # number of visits

def actionSpace(resolution):
    x,y = np.meshgrid(np.arange(0,1+resolution,resolution), np.arange(0,1+resolution,resolution))
    xy = np.concatenate((x.reshape(-1,1), y.reshape(-1,1)), axis=1)
    return xy.round(2).tolist()

#HyperParameters
epsilon = 0.1
gamma = 0.9
action_resolution = 0.1
episode_number = 3 #for submission this is fixed as 20


#Set-up
actions = actionSpace(action_resolution)
actionspace = range(len(actions)-1)
greedy_action = lambda s : max(actionspace, key=lambda a : Q[(s,a)])
max_q = lambda sp : max([Q[(sp,a)] for a in actionspace])

#Training of Q Table
for _ in range(episode_number):
    env.reset()
    nextstate = env.state
    while True:
        state = nextstate

        # Epsilon-Greedy
        if epsilon > random.random() :
            action = random.choice(actionspace)
            print('random_action',action)
        else :
            action = greedy_action(state)

        env_action = actions[action] #convert to ITN/IRS
        print('env_action', env_action)
        nextstate, reward, done, _ = env.evaluateAction(env_action)

        # Q-learning
        if done :
            Q[(state,action)] = Q[(state,action)] + 1./n[(state,action)] * ( reward - Q[(state,action)] )
            break
        else :
            Q[(state,action)] = Q[(state,action)] + 1./n[(state,action)] * ( reward + gamma * max_q(nextstate) - Q[(state,action)] )

#Greedy Policy Learnt from Q Table
best_policy = {state: list(actions[greedy_action(state-1)]) for state in range(1,6)}
best_reward = env.evaluatePolicy(best_policy)
print(best_policy, best_reward)

env_action [0.0, 0.0]
105  Evaluations Remaining
env_action [0.0, 0.0]
104  Evaluations Remaining
env_action [0.0, 0.0]
103  Evaluations Remaining
env_action [0.0, 0.0]
102  Evaluations Remaining
env_action [0.0, 0.0]
101  Evaluations Remaining
env_action [0.0, 0.0]
100  Evaluations Remaining
random_action 33
env_action [0.0, 0.3]
99  Evaluations Remaining
env_action [0.0, 0.0]
98  Evaluations Remaining
env_action [0.1, 0.0]
97  Evaluations Remaining
env_action [0.0, 0.0]
96  Evaluations Remaining
env_action [0.0, 0.0]
95  Evaluations Remaining
env_action [0.0, 0.3]
94  Evaluations Remaining
env_action [0.1, 0.0]
93  Evaluations Remaining
env_action [0.2, 0.0]
92  Evaluations Remaining
random_action 99
env_action [0.0, 0.9]
91  Evaluations Remaining
90  Evaluations Remaining
{1: [0.0, 0.0], 2: [0.0, 0.0], 3: [0.0, 0.3], 4: [0.2, 0.0], 5: [0.3, 0.0]} 12.936093964607444


### Creating a Valid Submission from Agent Code:

In [3]:
class Q_Agent():
    
    def __init__(self, environment):
        
        #Hyperparameters
        self.env = environment
        self.epsilon = 0.1
        self.gamma = 0.9
        self.action_resolution = 0.2
        self.Q = defaultdict(lambda : 0.) # Q-function
        self.n = defaultdict(lambda : 1.) # number of visits
        self.actions = actionSpace(self.action_resolution)
        self.actionspace = range(len(self.actions)-1)
        
    
    def actionSpace(self):
        x,y = np.meshgrid(np.arange(0,1+self.action_resolution,self.action_resolution), np.arange(0,1+self.action_resolution,self.action_resolution))
        xy = np.concatenate((x.reshape(-1,1), y.reshape(-1,1)), axis=1)
        return xy.round(2).tolist()

    def train(self):
        
        Q = self.Q
        n = self.n
        actions = self.actions
        actionspace = self.actionspace

        greedy_action = lambda s : max(actionspace, key=lambda a : Q[(s,a)])
        max_q = lambda sp : max([Q[(sp,a)] for a in actionspace])

        
        for _ in range(20): #Do not change
            
            self.env.reset()
            nextstate = self.env.state
            
            while True:
                state = nextstate

                # Epsilon-Greedy Action Selection
                if epsilon > random.random() :
                    action = random.choice(actionspace)
                else :
                    action = greedy_action(state)

                env_action = actions[action]#convert to ITN/IRS
                print('env_action', env_action)
                nextstate, reward, done, _ = self.env.evaluateAction(env_action)

                # Q-learning
                if done :
                    Q[(state,action)] = Q[(state,action)] + 1./n[(state,action)] * ( reward - Q[(state,action)] )
                    break
                else :
                    Q[(state,action)] = Q[(state,action)] + 1./n[(state,action)] * ( reward + gamma * max_q(nextstate) - Q[(state,action)] )

        return Q


    def generate(self):
        best_policy = None
        best_reward = -float('Inf')
        
        Q_trained = self.train()
        greedy_eval = lambda s : max(actionspace, key=lambda a : Q_trained[(s,a)])
        
        best_policy = {state: list(actions[greedy_eval(state-1)]) for state in range(1,6)}
        best_reward = self.env.evaluatePolicy(best_policy)
        
        print(best_policy, best_reward)
        
        return best_policy, best_reward

### Run the EvaluateChallengeSubmission Method with your Agent Class

In [4]:
EvaluateChallengeSubmission(ChallengeSeqDecEnvironment, Q_Agent, "Q_submission.csv")

env_action [0.0, 0.0]
105  Evaluations Remaining
env_action [0.0, 0.0]
104  Evaluations Remaining
env_action [0.0, 0.0]
103  Evaluations Remaining
env_action [0.6, 0.8]
102  Evaluations Remaining
env_action [0.6, 0.8]
101  Evaluations Remaining
env_action [0.0, 0.0]
100  Evaluations Remaining
env_action [0.0, 0.0]
99  Evaluations Remaining
env_action [0.0, 0.0]
98  Evaluations Remaining
env_action [0.6, 0.8]
97  Evaluations Remaining
env_action [0.0, 0.0]
96  Evaluations Remaining
env_action [0.0, 0.0]
95  Evaluations Remaining
env_action [0.0, 0.0]
94  Evaluations Remaining
env_action [0.0, 0.0]
93  Evaluations Remaining
env_action [0.6, 0.8]
92  Evaluations Remaining
env_action [0.2, 0.0]
91  Evaluations Remaining
env_action [0.0, 0.0]
90  Evaluations Remaining
env_action [0.0, 0.0]
89  Evaluations Remaining
env_action [0.0, 0.0]
88  Evaluations Remaining
env_action [0.6, 0.8]
87  Evaluations Remaining
env_action [0.2, 0.0]
86  Evaluations Remaining
env_action [0.0, 0.0]
85  Evaluati

env_action [0.4, 0.0]
36  Evaluations Remaining
env_action [1.0, 0.6]
35  Evaluations Remaining
env_action [0.8, 0.4]
34  Evaluations Remaining
env_action [0.6, 0.8]
33  Evaluations Remaining
env_action [0.2, 0.0]
32  Evaluations Remaining
env_action [0.4, 0.0]
31  Evaluations Remaining
env_action [1.0, 0.6]
30  Evaluations Remaining
env_action [0.8, 0.4]
29  Evaluations Remaining
env_action [0.6, 0.8]
28  Evaluations Remaining
env_action [0.2, 0.0]
27  Evaluations Remaining
env_action [0.4, 0.0]
26  Evaluations Remaining
env_action [1.0, 0.6]
25  Evaluations Remaining
env_action [0.8, 0.4]
24  Evaluations Remaining
env_action [0.6, 0.8]
23  Evaluations Remaining
env_action [0.2, 0.0]
22  Evaluations Remaining
env_action [0.4, 0.0]
21  Evaluations Remaining
env_action [1.0, 0.6]
20  Evaluations Remaining
env_action [0.8, 0.4]
19  Evaluations Remaining
env_action [0.6, 0.8]
18  Evaluations Remaining
env_action [0.2, 0.0]
17  Evaluations Remaining
env_action [0.4, 0.0]
16  Evaluations Re

env_action [0.4, 0.0]
70  Evaluations Remaining
env_action [0.8, 1.0]
69  Evaluations Remaining
env_action [0.4, 0.0]
68  Evaluations Remaining
env_action [0.2, 0.0]
67  Evaluations Remaining
env_action [0.4, 0.0]
66  Evaluations Remaining
env_action [0.4, 0.0]
65  Evaluations Remaining
env_action [0.8, 1.0]
64  Evaluations Remaining
env_action [0.4, 0.0]
63  Evaluations Remaining
env_action [0.2, 0.0]
62  Evaluations Remaining
env_action [0.4, 0.0]
61  Evaluations Remaining
env_action [0.4, 0.0]
60  Evaluations Remaining
env_action [0.8, 1.0]
59  Evaluations Remaining
env_action [0.4, 0.0]
58  Evaluations Remaining
env_action [0.0, 0.0]
57  Evaluations Remaining
env_action [0.4, 0.0]
56  Evaluations Remaining
env_action [0.4, 0.0]
55  Evaluations Remaining
env_action [0.8, 1.0]
54  Evaluations Remaining
env_action [0.4, 0.0]
53  Evaluations Remaining
env_action [0.0, 0.0]
52  Evaluations Remaining
env_action [0.4, 0.0]
51  Evaluations Remaining
env_action [0.4, 0.0]
50  Evaluations Re

env_action [0.0, 0.0]
104  Evaluations Remaining
env_action [0.0, 0.0]
103  Evaluations Remaining
env_action [0.2, 0.4]
102  Evaluations Remaining
env_action [0.0, 0.0]
101  Evaluations Remaining
env_action [0.4, 0.0]
100  Evaluations Remaining
env_action [0.2, 0.0]
99  Evaluations Remaining
env_action [0.0, 0.0]
98  Evaluations Remaining
env_action [0.2, 0.4]
97  Evaluations Remaining
env_action [0.2, 0.0]
96  Evaluations Remaining
env_action [0.4, 0.0]
95  Evaluations Remaining
env_action [0.4, 0.0]
94  Evaluations Remaining
env_action [0.0, 0.0]
93  Evaluations Remaining
env_action [0.2, 0.4]
92  Evaluations Remaining
env_action [0.4, 0.0]
91  Evaluations Remaining
env_action [0.4, 0.0]
90  Evaluations Remaining
env_action [0.4, 0.0]
89  Evaluations Remaining
env_action [0.0, 0.0]
88  Evaluations Remaining
env_action [0.2, 0.4]
87  Evaluations Remaining
env_action [0.4, 0.0]
86  Evaluations Remaining
env_action [0.0, 0.8]
85  Evaluations Remaining
env_action [0.4, 0.0]
84  Evaluatio

env_action [0.0, 0.8]
35  Evaluations Remaining
env_action [0.2, 0.4]
34  Evaluations Remaining
env_action [0.2, 0.0]
33  Evaluations Remaining
env_action [0.6, 0.8]
32  Evaluations Remaining
env_action [0.6, 0.0]
31  Evaluations Remaining
env_action [0.0, 0.8]
30  Evaluations Remaining
env_action [0.2, 0.0]
29  Evaluations Remaining
env_action [0.2, 0.0]
28  Evaluations Remaining
env_action [0.6, 0.8]
27  Evaluations Remaining
env_action [0.6, 0.0]
26  Evaluations Remaining
env_action [0.0, 0.8]
25  Evaluations Remaining
env_action [0.0, 0.6]
24  Evaluations Remaining
env_action [0.2, 0.0]
23  Evaluations Remaining
env_action [0.6, 0.8]
22  Evaluations Remaining
env_action [0.6, 0.0]
21  Evaluations Remaining
env_action [0.0, 0.8]
20  Evaluations Remaining
env_action [0.0, 0.6]
19  Evaluations Remaining
env_action [0.2, 0.0]
18  Evaluations Remaining
env_action [0.6, 0.8]
17  Evaluations Remaining
env_action [0.6, 0.0]
16  Evaluations Remaining
env_action [0.0, 0.8]
15  Evaluations Re

env_action [0.6, 0.2]
69  Evaluations Remaining
env_action [0.4, 0.0]
68  Evaluations Remaining
env_action [0.4, 0.0]
67  Evaluations Remaining
env_action [0.4, 0.0]
66  Evaluations Remaining
env_action [0.8, 0.0]
65  Evaluations Remaining
env_action [0.6, 0.2]
64  Evaluations Remaining
env_action [0.4, 0.0]
63  Evaluations Remaining
env_action [0.4, 0.0]
62  Evaluations Remaining
env_action [0.4, 0.0]
61  Evaluations Remaining
env_action [0.4, 0.8]
60  Evaluations Remaining
env_action [0.6, 0.2]
59  Evaluations Remaining
env_action [0.4, 0.0]
58  Evaluations Remaining
env_action [0.4, 0.0]
57  Evaluations Remaining
env_action [0.4, 0.4]
56  Evaluations Remaining
env_action [0.8, 0.0]
55  Evaluations Remaining
env_action [0.0, 0.0]
54  Evaluations Remaining
env_action [0.4, 0.0]
53  Evaluations Remaining
env_action [0.4, 0.0]
52  Evaluations Remaining
env_action [0.4, 0.4]
51  Evaluations Remaining
env_action [0.8, 0.0]
50  Evaluations Remaining
env_action [0.0, 0.0]
49  Evaluations Re